In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [1]:
import os
import sys
dir_path = 'C:/Users/gauth/Desktop/courses/Urban_computing(cs_5834)/assignment/assignment1/assignment_1/dataset'
sys.path.append(dir_path)

In [2]:
# some modules that need to be installed
!pip install geopandas==0.8.1 matplotlib==3.2.2 networkx==2.3  numpy==1.18.5  pandas==1.0.5  seaborn==0.10.1

  Using cached geopandas-0.8.1-py2.py3-none-any.whl (962 kB)
  Using cached matplotlib-3.2.2.tar.gz (40.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached networkx-2.3.zip (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-1.18.5.zip (5.4 MB)
  Installing build dependencies: started
 

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [68 lines of output]
      Running from numpy source directory.
      <string>:461: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      Cythonizing sources
      Processing numpy/random\_bounded_integers.pxd.in
      Processing numpy/random\mtrand.pyx
      C:\Users\gauth\AppData\Local\Temp\pip-install-a_c0b54c\numpy_cbc9057154cd402c912287512fa15bec\tools\cythonize.py:75: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
        required_version = LooseVersion('0.29.14')
      C:\Users\gauth\AppData\Local\Temp\pip-install-a_c0b54c\numpy_cbc9057154cd402c912287512fa15bec\tools\cythonize.py:77: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
        if LooseVersion(cython_version) < required_version:
      Pro

In [10]:
# including modules
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import datetime 
import seaborn as sns
import operator
import numpy as np
import geopandas as gp
%load_ext autoreload
%autoreload 2
import utils.airtraffic_helpers
import networkx as nx
import community
import random
from shapely.geometry import Point

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'gcd' from 'fractions' (C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1520.0_x64__qbz5n2kfra8p0\Lib\fractions.py)

In [ ]:
# Dometics airport details
airports_df = pd.read_csv("{}/dataset/288804893_T_MASTER_CORD.csv".format(dir_path))
print('Shape of the dataframe:',airports_df.shape,'/n')
print('Printing one record:',airports_df[:1].T)

In [ ]:
# Dometics airlines connections
trips_df = pd.read_csv("{}/dataset/288798530_T_T100D_MARKET_ALL_CARRIER.csv".format(dir_path))
print('Shape of the dataframe:',trips_df.shape,'/n')
print('Printing one record:',trips_df[:1].T)

In [ ]:
# Extracting edges - we consider a connection from one airport to another as an edge.
# Note: these edges will be directed.
edges = list(zip(trips_df['ORIGIN_AIRPORT_ID'],trips_df['DEST_AIRPORT_ID']))

In [ ]:
# creating directed and undirected graphs based on airports and their connections
G = nx.DiGraph()
G.add_edges_from(edges)
G_undirected = nx.Graph()
G_undirected.add_edges_from(edges)

In [ ]:
# printing the total number of nodes and edges of directed graph
print('Total number of airports:',len(list(G.nodes)))
print('Total number of connections:',len(list(G.edges)))

In [ ]:
# creating a GeoDataframe for plotting airports on a map
airport_ids = list(G.nodes)
edgelist = list(G.edges)
geo_stations = airtraffic_helpers.get_geodataframe_airports(airports_df,airport_ids)

In [ ]:
# reading the shape file of US
shp_us = gp.GeoDataFrame.from_file("{}/dataset/Igismap/Alabama_AL4_US_Poly.shp".format(dir_path))
shp_us=shp_us.to_crs({'init':'epsg:4326'})
shp_us.plot(figsize=(100,100),color='g',alpha=0.75)

In [ ]:
# plotting the nodes in the shape file of US
plt.style.use("default")
%matplotlib inline
fig, ax = plt.subplots(1,1,figsize=(50,50))
base = shp_us.plot(ax=ax, color='gray', alpha=0.2)
geo_stations.plot(ax=base, marker="o", color="r", markersize=10,alpha=0.8, zorder=0)
_ = ax.axis('off')
ax.set_title("Plot of airports in United States",fontsize=20)
fig.tight_layout()
fig.savefig("{}/figures/us_airports_nodes.pdf".format(dir_path))

In [ ]:
# displaying the network based on actual geographical placement of nodes
position_dict = {} # will contain the node coordinates 
for item in list(geo_stations.station_ids):
  position_dict[item] = geo_stations[geo_stations.station_ids==item].geometry.values[0].x,geo_stations[geo_stations.station_ids==item].geometry.values[0].y
fig, ax = plt.subplots(1,1,figsize=(100,40))
nx.draw(G,pos=position_dict)
fig.tight_layout()
fig.savefig("{}/figures/us_airports_network_withposition.pdf".format(dir_path))

In [ ]:
# displaying the network with optimal placement of nodes
airtraffic_helpers.plot_network(G,title="",edgealpha=0.08,node_dist=1,nodesize=20,savefig=False,figsize=(15,15))

## Strongly And Weakly Connected Components

In [ ]:
#Strongly Connected Components    
scc=[list(l) for l in nx.strongly_connected_components(G)]  #Strongly Connected Components
print("Number of Strongly Connected Components",len(scc),"/nSample Strongly Connected Components",scc[:3])
plt.plot(list(sorted(map(lambda x: len(x),scc),reverse=True)))
plt.xlabel("Number of Strongly Connected Components",fontsize=14)
plt.ylabel("Number of Nodes Per Component",fontsize=14)

In [ ]:
# Weakly Connected Components
wcc=[list(l) for l in nx.weakly_connected_components(G)]  #Strongly Connected Components
print("Number of Weakly Connected Components",len(wcc),"/nSample Weakly Connected Components",wcc[:3])
plt.plot(list(sorted(map(lambda x: len(x),wcc),reverse=True)))
plt.xlabel("Number of Weakly Connected Components",fontsize=18)
plt.ylabel("Number of Nodes Per Component",fontsize=18)

## Plot Degree Distribution

In [ ]:
# node degree calculation
node_indegrees=[item for item in dict(G.in_degree()).items()]
node_outdegrees=[item for item in dict(G.out_degree()).items()]
sorted_indegrees=sorted(node_indegrees,key=operator.itemgetter(1),reverse=True)
sorted_outdegrees=sorted(node_outdegrees,key=operator.itemgetter(1),reverse=True)

print("Sample Indegree List",node_indegrees[:5],"/n")
print("Sorted In Decreasing Order of Indegrees",sorted_indegrees[:5],"/n")
print("Sample Outdegree List",node_outdegrees[:5],"/n")
print("Sorted In Decreasing Order of Outdegree",sorted_outdegrees[:5],"/n")

node_degrees=airtraffic_helpers.getdegree(G)
print("Sample degree list",node_degrees[:5],"/n")
sorted_degrees=sorted(node_degrees,key=operator.itemgetter(1),reverse=True)
print("Sorted in decreasing order of degrees",sorted_degrees[:5],"/n")

In [ ]:
# plotting the degree distribution
sns.distplot(pd.Series(np.array(node_degrees).T[1], name="Degree distribution"))

In [ ]:
# plotting the degree-rank plot
airtraffic_helpers.generate_degree_rank_plot(edgelist)

# Network density, Connected Components, diameter and shortest paths.

In [ ]:
# find network density
print('Network density:',nx.density(G))

In [ ]:
sub_graphs = [G_undirected.subgraph(c).copy() for c in nx.connected_components(G_undirected)]
print('Number of connected components: ',len([k for k in sub_graphs]))

In [ ]:
# finding and plotting the giant connected component
Gc = max(sub_graphs, key=len)
Gc=nx.convert_node_labels_to_integers(Gc)
Gc.name='GCC'
print(nx.info(Gc))
print('Diameter of the Giant connected component:',nx.diameter(Gc))
print('Average shortest path:',nx.average_shortest_path_length(Gc))

In [ ]:
# plotting the GCC
airtraffic_helpers.plot_network(Gc,title="",edgealpha=0.08,node_dist=1,nodesize=20,savefig=False,figsize=(15,15))

## Centrality measures

In [ ]:
pd.Series(nx.degree_centrality(G)).sort_values().plot(kind='hist',bins=30,title="Degree centrality scores")

In [ ]:
pd.Series(nx.closeness_centrality(G)).sort_values().plot(kind='hist',bins=30,title="Closeness centrality scores")

In [ ]:
pd.Series(nx.betweenness_centrality(G)).sort_values().plot(kind='hist',bins=30,title="Betweenness centrality scores")

# Clustering coefficient

In [ ]:
# plotting average clustering coefficient
airtraffic_helpers.generate_clustering_coefficient_plot(G)
clust_coeff=nx.average_clustering(G)
print("Average Clustering Coefficient",round(clust_coeff,4))

# Node Centric Community Detection

In [ ]:
# As the network is large, we will randomly select 400 nodes for node centric community detection analysis
airportids_subset = random.sample(airport_ids,k=300)

In [ ]:
# filter edges to contain nodes only from the above generated airports subset
edgelist_subset = []
for item in edgelist:
  if item[0] in airportids_subset and item[1] in airportids_subset:
    edgelist_subset.append(item)

In [ ]:
G_subset_undirected = nx.Graph()
G_subset_undirected.add_edges_from(edgelist_subset)

In [ ]:
# identifying the cliques in the network
cl=nx.enumerate_all_cliques(G_subset_undirected)
#print last 10 cliques
print([l for l in cl][-10:])

In [ ]:
# printing the 5 largest cliques
print("5 Largest Cliques",sorted([l for l in nx.find_cliques(G_subset_undirected)],key=lambda x: len(x),reverse=True)[:5])

In [ ]:
# printing cliques containing Node ID 14107
print(nx.cliques_containing_node(G_subset_undirected,13495))

In [ ]:
# Find K cliques using percolation method. K=10
print([l for l in nx.algorithms.community.k_clique_communities(G_subset_undirected,10)])

In [ ]:
# finding k-core, in this case k=12
G_noselfloop = G_undirected
G_noselfloop.remove_edges_from(nx.selfloop_edges(G_noselfloop))
g_k=nx.k_core(G_noselfloop,k=12)
airtraffic_helpers.plot_network(g_k,node_dist=4,figsize=(12,12),nodesize=10)

In [ ]:
#Code to generate the shape file and station k_core visualization.
labels=g_k.nodes()
fig, ax = plt.subplots(1,1,figsize=(100,40))
base = shp_us.plot(ax=ax, color='gray', alpha=0.2)
geo_stations[geo_stations.station_ids.isin(labels)].plot(ax=base, marker="o", color="r", markersize=100,alpha=0.8, zorder=0)
_ = ax.axis('off')

# PAGERANK

In [ ]:
pg_rank=sorted([l for l in nx.pagerank(G).items()],key=lambda x: x[1],reverse=True)
print("Top 10 Stations By Pagerank",pg_rank[:10])

In [ ]:
hubs,authorities=nx.hits(G)
hubs=sorted([l for l in hubs.items()],key=lambda x: x[1],reverse=True)
authorities=sorted([l for l in authorities.items()],key=lambda x: x[1],reverse=True)
print("Top 10 Biggest Hubs",hubs[:10])
print("/nTop 10 Biggest Authorities",authorities[:10])

In [ ]:
# Top 10 pagerank nodes visualization
labels=list(map(lambda x: x[0],pg_rank[:10]))
fig, ax = plt.subplots(1,1,figsize=(100,40))
base = shp_us.plot(ax=ax, color='gray', alpha=0.2)
geo_stations[geo_stations.station_ids.isin(labels)].plot(ax=base, marker="o", color="r", markersize=100,alpha=0.8, zorder=0)
_ = ax.axis('off')
fig.tight_layout()
fig.savefig("{}/figures/pagerank.pdf".format(dir_path))

In [ ]:
# Top 10 hubs nodes visualization
labels=list(map(lambda x: x[0],hubs[:10]))
fig, ax = plt.subplots(1,1,figsize=(100,40))
base = shp_us.plot(ax=ax, color='gray', alpha=0.2)
geo_stations[geo_stations.station_ids.isin(labels)].plot(ax=base, marker="o", color="r", markersize=100,alpha=0.8, zorder=0)
_ = ax.axis('off')
fig.tight_layout()
fig.savefig("{}/figures/hubs.pdf".format(dir_path))

In [ ]:
# Top 10 authorities nodes visualization
labels=list(map(lambda x: x[0],authorities[:10]))
fig, ax = plt.subplots(1,1,figsize=(100,40))
base = shp_us.plot(ax=ax, color='gray', alpha=0.2)
geo_stations[geo_stations.station_ids.isin(labels)].plot(ax=base, marker="o", color="r", markersize=100,alpha=0.8, zorder=0)
_ = ax.axis('off')
fig.tight_layout()
fig.savefig("{}/figures/authorities.pdf".format(dir_path))